In [1]:
from environment import CoronagraphEnvironment

In [2]:
from gymnasium.envs.registration import register

register(
    id="Coronagraph-v0",
    entry_point="your_module:CoronagraphEnvironment",  # Replace with correct module path
)


In [3]:
e = CoronagraphEnvironment(num_modes = 4)
e._get_obs()

initializing coronagraph env. might take a minute.


{'image': OldStyleField([0., 0., 0., ..., 0., 0., 0.], shape=(57600,), dtype=float32),
 'slopes': OldStyleField([[-9.1864536e-07,  9.1164829e-06, -4.1101016e-06, ...,
                  4.1101016e-06, -9.1164829e-06,  9.1864536e-07],
                [-9.1864536e-07,  1.2017832e-05,  1.7143098e-05, ...,
                 -1.7143098e-05, -1.2017832e-05,  9.1864536e-07]],
               shape=(2, 1849), dtype=float32),
 'strehl': array([1.], dtype=float32)}

In [4]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.logger import configure

from environment import CoronagraphEnvironment  # Or import your env class directly

# Wrap in a function so it works cleanly
def make_env():
    return Monitor(CoronagraphEnvironment(num_modes=4))

# Vectorized environment (Stable-Baselines3 requires this)
env = DummyVecEnv([make_env])

# Set up logging
log_path = "./logs/"
new_logger = configure(log_path, ["stdout", "csv", "tensorboard"])

# Instantiate the PPO agent with a policy that handles Dict spaces
model = PPO(
    policy="MultiInputPolicy",
    env=env,
    verbose=1,
    tensorboard_log=log_path,
)

model.set_logger(new_logger)

# Optional: EvalCallback to evaluate the agent during training
eval_env = DummyVecEnv([make_env])
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path="./models/",
    log_path="./logs/",
    eval_freq=5000,
    deterministic=True,
    render=False,
)

# Train the model
model.learn(total_timesteps=100_000, callback=eval_callback)

# Save the final model
model.save("ppo_coronagraph_final")


initializing coronagraph env. might take a minute.
Logging to ./logs/
Using cuda device
initializing coronagraph env. might take a minute.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 0.000264 |
| time/              |          |
|    fps             | 3        |
|    iterations      | 1        |
|    time_elapsed    | 601      |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1          |
|    ep_rew_mean          | 0.00027    |
| time/                   |            |
|    fps                  | 3          |
|    iterations           | 2          |
|    time_elapsed         | 1198       |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00468318 |
|    clip_fraction        | 0.0158     |
|    clip_range           | 0

KeyboardInterrupt: 

In [ ]:
from stable_baselines3 import PPO

env = CoronagraphEnvironment()
model = PPO.load("ppo_coronagraph_final")

obs, _ = env.reset()
done = False

while not done:
    action, _ = model.predict(obs)
    obs, reward, terminated, truncated, info = env.step(action)
    env.render()
    done = terminated or truncated

In [ ]:
eval_env._get_obs()

AttributeError: 'DummyVecEnv' object has no attribute '_get_obs'